데이터가 적을 때 사용하는 평가방법 3가지
* 단순 홀드아웃 검증
* K겹 교차검증
* 셔플링을 사용한 반복 K겹 교차검증

In [ ]:
#홀드아웃 검증 구현 예
import numpy as np

num_validation_samples = 10000

np.random.shuffle(data)

validation_data = data[:num_validation_samples] #검증 세트 만들기
data = data[num_validation_samples:]

training_data = data[:] #훈련세트 만들기

model=get_model()
model.train(training_data)
validation_score = model.evaluate(validation_data)

# 여기에서 모델을 튜닝하고,
# 다시 훈련하고, 평가하고, 또 다시 튜닝하고... 

model = get_model()
model.train(np.concatenate([training_data, 
                            validation_data])) #하이퍼파라미터 튜닝이 끝나면 테스트데이터를 제외한 모든 데이터를 사용해 모델을 다시 다시 훈련
test_score = model.evaluate(test_data)

홀드아웃 검증방법에서는 데이터의 일정량을 테스트 세트로 떼어놓고 남은 데이터로 훈련하고 테스트 세트로 평가한다.
* 데이터가 적을 때는 검증세트와 테스트세트 샘플이 너무 적어 주어진 전체 데이터를 통계적으로 대표하지 못할 수 있다. 

=>다른 난수 초깃값으로 셔플링해서 데이터를 나눌때 모델 성능이 매우 달라지면 이 문제를 가진 것

In [ ]:
#K겹 교차검증 구현 예
k = 4
num_validation_samples = len(data) // k

np.random.shuffle(data)

validation_scores = []
for fold in range(k):
    
    validation_data = data[num_validation_samples * fold: num_validation_samples * (fold + 1)]
    training_data = data[:num_validation_samples * fold] + 
    data[num_validation_samples * (fold + 1):]

    model = get_model()
    model.train(training_data)
    validation_score = model.evaluate(validation_scores)
    validaton_scores.append(validation_score)

    validation_score = np.average(validation_scores)

    model = get_model()
    model.train(data)
    test_score = model.evaluate(test_data)

셔플링을 사용한 반복 K겹 교차검증: 가용데이터가 적고 가능한 정확히 모델을 평가하려 할 때 사용. K겹 교차검증을 여러번 적용하되 K개의 분할로 나누기 전에 매번 데이터를 무작위로 섞는다. 결국 PxK개의 모델(P는 반복횟수)을 훈련하고 평가한다

영화리뷰 분류 모델 네트워크 크기 축소

In [10]:
# 원본 모델
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [11]:
# 작은 용량의 모델
model = models.Sequential()
model.add(layers.Dense(6, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(6, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

작은 네트워크가 더 나중에 과대적합되기 시작한다

In [12]:

# 큰 용량의 모델
model = models.Sequential()
model.add(layers.Dense(1024, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

첫 번째 에포크 이후 거의 바로 과대적합이 시작되고 검증손실도 매우 불안정. 훈련 손실은 매우 빠르게 0에 가까워지지만 더욱 과대적합에 민감해짐

간단한 모델이 복잡한 모델보다 덜 과대적합될 가능성이 높고 간단한 모델은 적은 수의 파라미터를 가진 모델, 파라미터 값 분포 엔트로피가 작은 모델을 말함. 네트워크 복잡도에 제한을 둬서 가중치가 작은값을 가지도록 강제하는 것을 가중치 규제라고 하고 이에는 두가지 형태가 있다

* L1규제: 가중치의 절댓값에 비례하는 비용이 추가됨
* L2규제: 가중치의 제곱에 비레하는 비용이 추가되고 가중치 감쇠라고도 부름

In [13]:
# 모델에 L2 가중치 추가하기
from keras import regularizers

model = models.Sequential()
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                       activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                       activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

과대적합에 훨씬 잘 견딘다 /  l2(0.001)은 가중치 행렬의 모든 원소를 제곱하고 0.001을 곱해서 네트워크 전체 손실에 더해진다는 의미이다

In [14]:
# 케라스에서 사용할 수 있는 가중치 규제
from keras import regularizers

regularizers.l1(0.001) # L1 규제
regularizers.l1_l2(l1=0.001, l2=0.001) # L1 L2 규제 병행

드롭아웃:훈련하는 동안 무작위로 층의 일부 훈련 특성을 제외시킨다(0으로 만듬). 드롭아웃 비율은 0.2~0.5로 테스트단계에서는 드롭아웃되지 않지만 훈련할 때보다 더 많은 유닛이 활성화 되기 때문에 층의 출력을 드롭아웃 비율에 비례해 줄여줌
* 층의 추력 값에 노이즈를 추가해 중요하지 ㅇ

In [15]:
# IMDB 네트워크에 드롭아웃 추가하기
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))